In [2]:
pip install pandahouse

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandahouse: filename=pandahouse-0.2.7-py2.py3-none-any.whl size=5912 sha256=257e08a77613a14e607ad62356f7da5e2fd13e7de1eadf1834ddaadc230ed0ec
  Stored in directory: c:\users\pazun\appdata\local\pip\cache\wheels\2b\75\4a\22824efa98a925fce4a1f2c18e85b55eb75179dfb9e9d59f45
Successfully built pandahouse
Note: you may need to restart the kernel to use updated packages.


In [1]:
#импортируем нужные либы
import pandahouse as ph
import pandas as pd

In [2]:
#объявляем параметры подключения
connection = dict(database='test',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student-rw',
                  password='656e2b0c9c')

In [5]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
SELECT *
FROM test.installs
limit 1000
"""
df = ph.read_clickhouse(query, connection=connection)
df

,InstallationDate,InstallCost,Platform,DeviceID,Source
0,2019-03-02,0,android,7950068545577019282,Source_27
1,2019-03-17,49,android,17173992779193729517,Source_14
2,2019-04-07,56,android,9528182466778893591,Source_14
3,2019-06-25,39,android,2212531864415574595,Source_9
4,2019-04-13,0,android,6959033924999748551,Source_27
...,...,...,...,...,...
995,2019-01-09,334,android,5435327797863630841,Source_14
996,2019-03-05,19,iOS,16012086443778979967,Source_9
997,2019-06-12,0,iOS,1637578224854940304,Source_27
998,2019-07-01,171,android,9176486286906828772,Source_9


In [6]:
#выгружаем данные из pandas в excel
df.to_excel("./ch_data.xlsx", sheet_name='Data', index=False, encoding='UTF-8')

In [7]:
#читаем данные из csv в pandas
df_in = pd.read_csv("./medium.csv", sep='\t')
df_in

,published,domain,url
0,2012-08-13 22:54:53.510Z,medium.com,https://medium.com/policy/medium-terms-of-serv...
1,2012-08-13 22:57:17.248Z,medium.com,https://medium.com/policy/medium-privacy-polic...
2,2016-11-04 23:40:43.364Z,medium.com,https://medium.com/@Medium/personalize-your-me...
3,2016-12-24 18:21:13.427Z,medium.com,https://medium.com/holiday-poems/xmas-morning-...
4,2015-09-22 21:37:48.207Z,blog.medium.com,https://blog.medium.com/taking-a-side-on-net-n...
...,...,...,...
92186,2017-05-23 17:15:09.193Z,medium.com,https://medium.com/@KyleAndrews1994/its-time-t...
92187,2017-01-31 09:59:12.316Z,medium.com,https://medium.com/silent-protagonist/next-wee...
92188,2017-02-16 14:15:20.116Z,medium.com,https://medium.com/@utapartment26/there-used-t...
92189,2017-06-27 10:16:21.668Z,medium.com,https://medium.com/silent-protagonist/the-snes...


In [9]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query_create = """
CREATE TABLE test.a_pos_import_test (
    published String,
    domain String,
    url String
)
ENGINE = Log
"""
df_create = ph.read_clickhouse(query_create, connection=connection)
df_create

ClickhouseException: b'Code: 57. DB::Exception: Table test.a_pos_import_test already exists. (TABLE_ALREADY_EXISTS) (version 21.12.2.17 (official build))\n'

In [10]:
#записываем данные из pandas в clickhouse
ph.to_clickhouse(df_in, 'beslan_import_test', index=False, connection=connection)

92191

In [11]:
#проверяем, что записалось
query_test = """
SELECT *
FROM test.beslan_import_test
"""
df_new = ph.read_clickhouse(query_test, connection=connection)
df_new

,published,domain,url
0,2016-11-09 13:12:26.911Z,medium.com,https://medium.com/@raphael.carvalheira/como-s...
1,2016-11-09 13:15:45.000Z,medium.com,https://medium.com/@raphael.carvalheira/ensaio...
2,2016-11-04 14:01:03.415Z,medium.com,https://medium.com/starts-with-a-bang/why-does...
3,2015-09-22 15:05:21.254Z,medium.com,https://medium.com/laguna-serpiente/apuntes-pa...
4,2016-01-11 15:01:11.442Z,medium.com,https://medium.com/laguna-serpiente/brujer%C3%...
...,...,...,...
2635067,2017-02-20 11:08:13.000Z,medium.com,https://www.cold-steel.org/2017/ss-gb-review/
2635068,2017-05-08 21:08:59.838Z,medium.com,https://medium.com/@brupaese/como-podemos-ajud...
2635069,2017-05-08 12:14:22.198Z,medium.com,https://medium.com/copyright-untangled/teacher...
2635070,2017-03-29 13:23:15.183Z,medium.com,https://medium.com/copyright-untangled/launche...
